# Tool-Enabled Agents with Human Feedback

## Overview

This notebook demonstrates how to build a sophisticated workflow that combines:
- **Tool/function calling**: Agents that invoke Python functions to gather information
- **Human-in-the-loop**: Pausing execution to collect human feedback
- **Multi-agent coordination**: Writer agent → Human review → Editor agent

The workflow creates marketing copy through a three-stage process:
1. **Writer Agent**: Calls tools to gather product details and brand guidelines, then drafts initial copy
2. **Human Reviewer**: Reviews the draft and provides feedback via `RequestInfoExecutor`
3. **Final Editor Agent**: Incorporates human feedback to produce polished output

## Key Concepts

### Tool Calling
- Attach Python functions to agents via the `tools` parameter
- Use `ToolMode.REQUIRED_ANY` to force the agent to call tools before responding
- Monitor tool calls and results via `FunctionCallContent` and `FunctionResultContent`

### Human-in-the-Loop Pattern
- `RequestInfoExecutor`: Built-in executor that pauses workflow for external input
- `RequestInfoEvent`: Emitted when human input is needed
- `RequestResponse`: Packages human feedback for re-injection into the workflow

### Custom Coordinator Executor
- `DraftFeedbackCoordinator`: Bridges between agents and human feedback
- Preserves full conversation history (including tool traces)
- Routes messages between writer, human, and editor

## Pipeline Layout

```
Writer Agent (with tools)
    ↓
DraftFeedbackCoordinator
    ↓
RequestInfoExecutor (human feedback)
    ↓
DraftFeedbackCoordinator
    ↓
Final Editor Agent
```

## Prerequisites

- Azure OpenAI configured with environment variables:
  - `AZURE_OPENAI_API_KEY`
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_DEPLOYMENT_NAME`
  - `AZURE_OPENAI_API_VERSION`
- Azure CLI authentication: Run `az login` before executing
- Agent Framework installed: `pip install agent-framework`

## Setup and Imports

In [ ]:
import asyncio
import json
from dataclasses import dataclass, field
from typing import Annotated

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentExecutorRequest,
    AgentExecutorResponse,
    AgentRunUpdateEvent,
    ChatMessage,
    Executor,
    FunctionCallContent,
    FunctionResultContent,
    RequestInfoEvent,
    RequestInfoExecutor,
    RequestInfoMessage,
    RequestResponse,
    Role,
    ToolMode,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import Field
# Load environment variables from .env file
load_dotenv('../../.env')


## Define Tool Functions

These Python functions will be available to the writer agent as tools. The agent can call them to gather factual information before drafting copy.

### Key Points:
- Use `Annotated` type hints with `Field(description=...)` to provide parameter documentation
- Return string values that the agent can incorporate into its reasoning
- Keep functions simple and focused on data retrieval

In [ ]:
def fetch_product_brief(
    product_name: Annotated[str, Field(description="Product name to look up.")],
) -> str:
    """Return a marketing brief for a product."""
    briefs = {
        "lumenx desk lamp": (
            "Product: LumenX Desk Lamp\n"
            "- Three-point adjustable arm with 270° rotation.\n"
            "- Custom warm-to-neutral LED spectrum (2700K-4000K).\n"
            "- USB-C charging pad integrated in the base.\n"
            "- Designed for home offices and late-night study sessions."
        )
    }
    return briefs.get(product_name.lower(), f"No stored brief for '{product_name}'.")


def get_brand_voice_profile(
    voice_name: Annotated[str, Field(description="Brand or campaign voice to emulate.")],
) -> str:
    """Return guidance for the requested brand voice."""
    voices = {
        "lumenx launch": (
            "Voice guidelines:\n"
            "- Friendly and modern with concise sentences.\n"
            "- Highlight practical benefits before aesthetics.\n"
            "- End with an invitation to imagine the product in daily use."
        )
    }
    return voices.get(voice_name.lower(), f"No stored voice profile for '{voice_name}'.")

## Define Request Message Type

This dataclass defines the structure of feedback requests sent to the human reviewer.

In [ ]:
@dataclass
class DraftFeedbackRequest(RequestInfoMessage):
    """Payload sent to RequestInfoExecutor for human review."""

    prompt: str = ""
    draft_text: str = ""
    conversation: list[ChatMessage] = field(default_factory=list)  # type: ignore[reportUnknownVariableType]

## Create Custom Coordinator Executor

The `DraftFeedbackCoordinator` acts as a bridge between the writer agent, human feedback, and final editor.

### Handler Methods:

#### 1. `on_writer_response`
- Receives the draft from the writer agent
- Preserves full conversation history (including tool calls)
- Sends `DraftFeedbackRequest` to `RequestInfoExecutor`

#### 2. `on_human_feedback`
- Receives human feedback via `RequestResponse`
- Reconstructs conversation with feedback injected
- Forwards to final editor with instructions to incorporate feedback

In [ ]:
class DraftFeedbackCoordinator(Executor):
    """Bridge between the writer agent, human feedback, and final editor."""

    def __init__(self, *, id: str = "draft_feedback_coordinator") -> None:
        super().__init__(id)

    @handler
    async def on_writer_response(
        self,
        draft: AgentExecutorResponse,
        ctx: WorkflowContext[DraftFeedbackRequest],
    ) -> None:
        # Preserve the full conversation so the final editor can see tool traces and the initial prompt.
        conversation: list[ChatMessage]
        if draft.full_conversation is not None:
            conversation = list(draft.full_conversation)
        else:
            conversation = list(draft.agent_run_response.messages)
        draft_text = draft.agent_run_response.text.strip()
        if not draft_text:
            draft_text = "No draft text was produced."

        prompt = (
            "Review the draft from the writer and provide a short directional note "
            "(tone tweaks, must-have detail, target audience, etc.). "
            "Keep it under 30 words."
        )
        await ctx.send_message(DraftFeedbackRequest(prompt=prompt, draft_text=draft_text, conversation=conversation))

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[DraftFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        note = (feedback.data or "").strip()
        request = feedback.original_request

        conversation: list[ChatMessage] = list(request.conversation)
        instruction = (
            "A human reviewer shared the following guidance:\n"
            f"{note or 'No specific guidance provided.'}\n\n"
            "Rewrite the draft from the previous assistant message into a polished final version. "
            "Keep the response under 120 words and reflect any requested tone adjustments."
        )
        conversation.append(ChatMessage(Role.USER, text=instruction))
        await ctx.send_message(AgentExecutorRequest(messages=conversation, should_respond=True))

## Build and Execute Workflow

### Workflow Construction:

1. **Create chat client** with Azure CLI authentication
2. **Create writer agent** with tools and `ToolMode.REQUIRED_ANY`
3. **Create editor agent** for final polishing
4. **Create coordinator** to orchestrate feedback flow
5. **Create RequestInfoExecutor** for human input
6. **Build workflow** with edges defining the pipeline

### Execution Loop:

- Stream workflow events asynchronously
- Monitor `AgentRunUpdateEvent` for tool calls and agent outputs
- Capture `RequestInfoEvent` for human feedback prompts
- Continue with `send_responses_streaming` after collecting feedback

In [ ]:
async def run_workflow() -> None:
    """Run the workflow and bridge human feedback between two agents."""
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )

    writer_agent = chat_client.create_agent(
        name="writer_agent",
        instructions=(
            "You are a marketing writer. Call the available tools before drafting copy so you are precise. "
            "Always call both tools once before drafting. Summarize tool outputs as bullet points, then "
            "produce a 3-sentence draft."
        ),
        tools=[fetch_product_brief, get_brand_voice_profile],
        tool_choice=ToolMode.REQUIRED_ANY,
    )

    final_editor_agent = chat_client.create_agent(
        name="final_editor_agent",
        instructions=(
            "You are an editor who polishes marketing copy using human guidance. "
            "Respect factual details from the prior messages while applying the feedback."
        ),
    )

    feedback_coordinator = DraftFeedbackCoordinator()
    request_info_executor = RequestInfoExecutor(id="human_feedback")

    workflow = (
        WorkflowBuilder()
        .add_agent(writer_agent, id="Writer")
        .add_agent(final_editor_agent, id="FinalEditor", output_response=True)
        .set_start_executor(writer_agent)
        .add_edge(writer_agent, feedback_coordinator)
        .add_edge(feedback_coordinator, request_info_executor)
        .add_edge(request_info_executor, feedback_coordinator)
        .add_edge(feedback_coordinator, final_editor_agent)
        .build()
    )

    print(
        "Interactive mode. When prompted, provide a short feedback note for the editor (type 'exit' to quit).",
        flush=True,
    )

    pending_responses: dict[str, str] | None = None
    completed = False
    printed_tool_calls: set[str] = set()
    printed_tool_results: set[str] = set()

    while not completed:
        last_executor: str | None = None
        stream = (
            workflow.send_responses_streaming(pending_responses)
            if pending_responses is not None
            else workflow.run_stream(
                "Create a short launch blurb for the LumenX desk lamp. Emphasize adjustability and warm lighting."
            )
        )
        pending_responses = None
        requests: list[tuple[str, DraftFeedbackRequest]] = []

        async for event in stream:
            if isinstance(event, AgentRunUpdateEvent):
                executor_id = event.executor_id
                update = event.data
                # Extract and print any new tool calls or results from the update.
                function_calls = [c for c in update.contents if isinstance(c, FunctionCallContent)]  # type: ignore[union-attr]     
                function_results = [c for c in update.contents if isinstance(c, FunctionResultContent)]  # type: ignore[union-attr] 
                if executor_id != last_executor:
                    if last_executor is not None:
                        print()
                    print(f"{executor_id}:", end=" ", flush=True)
                    last_executor = executor_id
                # Print any new tool calls before the text update.
                for call in function_calls:
                    if call.call_id in printed_tool_calls:
                        continue
                    printed_tool_calls.add(call.call_id)
                    args = call.arguments
                    if isinstance(args, dict):
                        args_preview = json.dumps(args, ensure_ascii=False)
                    else:
                        args_preview = (args or "").strip()
                    print(
                        f"\n{executor_id} [tool-call] {call.name}({args_preview})",
                        flush=True,
                    )
                    print(f"{executor_id}:", end=" ", flush=True)
                # Print any new tool results before the text update.
                for result in function_results:
                    if result.call_id in printed_tool_results:
                        continue
                    printed_tool_results.add(result.call_id)
                    result_text = result.result
                    if not isinstance(result_text, str):
                        result_text = json.dumps(result_text, ensure_ascii=False)
                    print(
                        f"\n{executor_id} [tool-result] {result.call_id}: {result_text}",
                        flush=True,
                    )
                    print(f"{executor_id}:", end=" ", flush=True)
                # Finally, print the text update.
                print(update, end="", flush=True)
            elif isinstance(event, RequestInfoEvent) and isinstance(event.data, DraftFeedbackRequest):
                # Stash the request so we can prompt the human after the stream completes.
                requests.append((event.request_id, event.data))
                last_executor = None
            elif isinstance(event, WorkflowOutputEvent):
                last_executor = None
                response = event.data
                print("\n===== Final output =====")
                final_text = getattr(response, "text", str(response))
                print(final_text.strip())
                completed = True

        if requests and not completed:
            responses: dict[str, str] = {}
            for request_id, request in requests:
                print("\n----- Writer draft -----")
                print(request.draft_text.strip())
                print("\nProvide guidance for the editor (or press Enter to accept the draft).")
                answer = input("Human feedback: ").strip()  # noqa: ASYNC250
                if answer.lower() == "exit":
                    print("Exiting...")
                    return
                responses[request_id] = answer
            pending_responses = responses

    print("Workflow complete.")

## Run the Workflow

In [ ]:
await run_workflow()

## Expected Output

```
Interactive mode. When prompted, provide a short feedback note for the editor (type 'exit' to quit).
Writer: [tool-call] fetch_product_brief({"product_name": "lumenx desk lamp"})
Writer: [tool-result] call_xxx: Product: LumenX Desk Lamp...
Writer: [tool-call] get_brand_voice_profile({"voice_name": "lumenx launch"})
Writer: [tool-result] call_yyy: Voice guidelines...
Writer: Meet the LumenX Desk Lamp—your perfect companion for focused work...

----- Writer draft -----
Meet the LumenX Desk Lamp—your perfect companion for focused work...

Provide guidance for the editor (or press Enter to accept the draft).
Human feedback: Make it more energetic and emphasize the USB charging

FinalEditor: Transform your workspace with the LumenX Desk Lamp! ...

===== Final output =====
Transform your workspace with the LumenX Desk Lamp! ...
Workflow complete.
```

## Key Takeaways

### 1. Tool Integration
- Attach functions to agents via `tools=[...]`
- Use `ToolMode.REQUIRED_ANY` to enforce tool usage
- Monitor `FunctionCallContent` and `FunctionResultContent` in updates

### 2. Human-in-the-Loop Pattern
- `RequestInfoExecutor` provides built-in pause-for-input capability
- `RequestInfoEvent` signals when input is needed
- `send_responses_streaming()` resumes workflow with collected responses

### 3. Conversation Preservation
- Save `full_conversation` to retain tool traces
- Pass complete history to downstream agents for context
- Enables the editor to see *how* the writer reached its conclusion

### 4. Custom Coordinator Pattern
- Create specialized executors to orchestrate complex flows
- Use multiple `@handler` methods to process different message types
- Type hints on `WorkflowContext` determine routing behavior

### 5. Interactive Streaming
- Process events asynchronously with `async for`
- Display incremental updates for better UX
- Collect feedback requests and resume with `send_responses_streaming()`